In [ ]:
import os,sys
import h5py  
import numpy as np
import math,cmath
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import curve_fit, fsolve
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams.update({"axes.grid" : True})

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

ens='cA211.53.24'
lat=yu.LatticeEnsemble(ens)

In [ ]:
path='/project/s1174/lyan/code/projectData/nucleon_sigma_term/cA211.53.24/test_NJNpi_GEVP_noPost/'
templateMomListPath=path+'/templateFiles/templateMomList.h5'
with h5py.File(templateMomListPath) as f:
    momList=f['B'][:,9:12]
momDic={}
for i in range(len(momList)):
    momDic[tuple(momList[i])]=i

tfList=[10,12,14]

In [ ]:
data={}
pi2Str='pi2=0_0_1'; mom=momDic[(0,0,1)]; Gc=0; ab=0; preFactor=1
for diag in ['B','W','Z']:
    data[diag]={}
    with h5py.File(path+'avg-merge/'+diag+'.h5') as f:
        conts=[]
        for cont in f['0000/12'][pi2Str].keys():
            conts.append(cont.split('_')[0])
        conts=set(conts)
        for cont in conts:
            data[diag][cont]=np.array(
                [np.array([f[cfg]['12'][pi2Str][cont+'_deltat_'+str(tf)][1:tf,mom,Gc,ab] for tf in tfList]) for cfg in f.keys()]
            )

In [ ]:
for diag in data.keys():
    for cont in data[diag].keys():
        dat=data[diag][cont]

        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.array([np.real(preFactor*(t[i_tf][:])) for i_tf in range(len(tfList))],dtype=object)
            return t

        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        fig,axs=yu.getFigAxs(1,1)
        fig.suptitle(cont)
        irow=0;icol=0
        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim1)

# T & D1ii

checking all mom zero case, real part, supposed to be equal.
I missed a sign for pion's gamma conj, so here should be opposite-equal.

In [ ]:
path='/project/s1174/lyan/code/projectData/nucleon_sigma_term/cA211.53.24/test_NJNpi_GEVP_noPost/avg-merge/'
# path='/project/s1174/lyan/code/projectData/nucleon_sigma_term/cA211.53.24/test2/avg-merge/'
 
pi2StrT='pi2=0_0_1'; momT=momDicT[(0,-1,1)]; abT=0  # pf1
pi2StrD='pi2=0_0_0'; momD=momDicD[(0,-1,0,0,0,1)]; abD=0 # pf1 pf2

In [ ]:
fig, axs = getFigAxs(1,1,Lrow=16,Lcol=20)
tmin=0; tmax=16
irow=0; icol=0

spacing=0.04
i=0
for cont in ['Tseq11', 'Tseq12','Tseq13','Tseq14']:
# for cont in ['Tseq11', 'Tseq12', 'Tseq13', 'Tseq14', 'Tseq21', 'Tseq22', 'Tseq23', 'Tseq24', 'Tseq25', 'Tseq26']:
    with h5py.File(path+'T.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2StrT][cont][:,momT,0,abT] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

for cont in ['D1ii13', 'D1ii14','D1ii15','D1ii16']:
# for cont in ['D1ii1', 'D1ii10', 'D1ii13', 'D1ii14', 'D1ii15', 'D1ii16', 'D1ii2', 'D1ii3', 'D1ii4', 'D1ii9']:
    with h5py.File(path+'D1ii.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2StrD][cont][:,momD,0,abD] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1
        
t=max(abs(np.array(axs[irow,icol].get_ylim())))
axs[irow,icol].set_ylim([-t,t])
axs[irow,icol].legend()
axs[irow,icol].grid()


In [ ]:
fig, axs = getFigAxs(1,1,Lrow=16,Lcol=20)
tmin=0; tmax=16
irow=0; icol=0

spacing=0.04
i=0
for cont in ['Tseq25', 'Tseq26']:
# for cont in ['Tseq11', 'Tseq12', 'Tseq13', 'Tseq14', 'Tseq21', 'Tseq22', 'Tseq23', 'Tseq24', 'Tseq25', 'Tseq26']:
    with h5py.File(path+'T.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2StrT][cont][:,momT,0,abT] for cfg in list(f.keys())])
        # dat=np.abs(dat)
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

for cont in ['D1ii9', 'D1ii10']:
# for cont in ['D1ii1', 'D1ii10', 'D1ii13', 'D1ii14', 'D1ii15', 'D1ii16', 'D1ii2', 'D1ii3', 'D1ii4', 'D1ii9']:
    with h5py.File(path+'D1ii.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2StrD][cont][:,momD,0,abD] for cfg in list(f.keys())])
        # dat=np.abs(dat)
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

t=max(abs(np.array(axs[irow,icol].get_ylim())))
# axs[irow,icol].set_ylim([-t,t])
axs[irow,icol].legend()
axs[irow,icol].grid()
# axs[irow,icol].set_yscale('log')

In [ ]:
fig, axs = getFigAxs(1,1,Lrow=16,Lcol=20)
tmin=0; tmax=16
irow=0; icol=0

spacing=0.04
i=0
for cont in ['Tseq21', 'Tseq22','Tseq23','Tseq24']:
# for cont in ['Tseq11', 'Tseq12', 'Tseq13', 'Tseq14', 'Tseq21', 'Tseq22', 'Tseq23', 'Tseq24', 'Tseq25', 'Tseq26']:
    with h5py.File(path+'T.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2StrT][cont][:,momT,0,abT] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

for cont in ['D1ii1', 'D1ii2','D1ii3','D1ii4']:
# for cont in ['D1ii1', 'D1ii10', 'D1ii13', 'D1ii14', 'D1ii15', 'D1ii16', 'D1ii2', 'D1ii3', 'D1ii4', 'D1ii9']:
    with h5py.File(path+'D1ii.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2StrD][cont][:,momD,0,abD] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

t=max(abs(np.array(axs[irow,icol].get_ylim())))
axs[irow,icol].set_ylim([-t,t])
axs[irow,icol].legend()
axs[irow,icol].grid()


In [ ]:
# # u vs. d

# fig, axs = getFigAxs(1,1,Lrow=16,Lcol=20)
# tmin=0; tmax=16
# irow=0; icol=0

# spacing=0.04
# i=0
# with h5py.File(path+'T.h5') as f:
#     dat=np.imag([
#         np.sum([f[cfg+'/12/'+pi2StrT][cont][:,momT,0,abT] for cont in ['Tseq21', 'Tseq22','Tseq23','Tseq24']],axis=0)
#             for cfg in f.keys()])
#     (mean,cov,err) = yu.jackknife(dat)
#     tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
#     axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label='u')
#     t=np.mean(dat,axis=0)
#     print(cont)
#     print(t)
# i+=1

# with h5py.File(path+'T.h5') as f:
#     dat=np.imag([
#         np.sum([f[cfg+'/12/'+pi2StrT][cont][:,momT,0,abT] for cont in ['Tseq25', 'Tseq26']],axis=0)
#             for cfg in f.keys()])
#     (mean,cov,err) = yu.jackknife(dat)
#     tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
#     axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label='d')
#     t=np.mean(dat,axis=0)
#     print(cont)
#     print(t)
# i+=1


# # for cont in ['D1ii1', 'D1ii2','D1ii3','D1ii4']:
# # # for cont in ['D1ii1', 'D1ii10', 'D1ii13', 'D1ii14', 'D1ii15', 'D1ii16', 'D1ii2', 'D1ii3', 'D1ii4', 'D1ii9']:
# #     with h5py.File(path+'D1ii.h5') as f:
# #         dat=np.real([f[cfg+'/12/'+pi2StrD][cont][:,momD,0,abD] for cfg in f.keys()])/12*(-1)
# #         (mean,cov,err) = yu.jackknife(dat)
# #         tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
# #         axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white',label=cont)
# #         t=np.mean(dat,axis=0)
# #         print(cont)
# #         print(t)
# #     i+=1

# # t=max(abs(np.array(axs[irow,icol].get_ylim())))
# axs[irow,icol].set_ylim([-0.2e-9,0])
# axs[irow,icol].legend()
# axs[irow,icol].grid()

# W

In [ ]:
path='/project/s1174/lyan/code/projectData/nucleon_sigma_term/cA211.53.24/test_NJNpi_GEVP_noPost/avg-merge/'

templateMomListPath='/project/s1174/lyan/code/projectData/nucleon_sigma_term/cA211.53.24/test_NJNpi_GEVP_noPost/templateFiles/templateMomList.h5'
with h5py.File(templateMomListPath) as f:
    momList=f['B_2pt'][:,3:9]
momDic={}
for i in range(len(momList)):
    momDic[tuple(momList[i])]=i

pi2Str='pi2=0_0_0'; mom=momDic[(0,0,1,0,0,0)] # pf1 pf2

In [ ]:
fig, axs = getFigAxs(1,1,Lrow=16,Lcol=20)
tmin=0; tmax=16
irow=0; icol=0

spacing=0.04
i=0
for cont in ['W29','W30','W31','W32']:
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

for cont in ['W17','W18','W19','W20']:
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

t=max(abs(np.array(axs[irow,icol].get_ylim())))
axs[irow,icol].set_ylim([-t,t])
axs[irow,icol].legend()
axs[irow,icol].grid()


In [ ]:
fig, axs = getFigAxs(1,1,Lrow=16,Lcol=20)
tmin=0; tmax=16
irow=0; icol=0

spacing=0.04
i=0
for cont in ['W33','W34','W35','W36']:
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

for cont in ['W21','W22','W23','W24']:
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

t=max(abs(np.array(axs[irow,icol].get_ylim())))
axs[irow,icol].set_ylim([-t,t])
axs[irow,icol].legend()
axs[irow,icol].grid()


In [ ]:
fig, axs = getFigAxs(1,1,Lrow=16,Lcol=20)
tmin=0; tmax=16
irow=0; icol=0

spacing=0.04
i=0
for cont in ['W25','W26','W27','W28']:
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

for cont in ['W5','W6','W7','W8',]:
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

t=max(abs(np.array(axs[irow,icol].get_ylim())))
axs[irow,icol].set_ylim([-t,t])
axs[irow,icol].legend()
axs[irow,icol].grid()


In [ ]:
fig, axs = getFigAxs(1,1,Lrow=16,Lcol=20)
tmin=0; tmax=16
irow=0; icol=0

spacing=0.04
i=0
for cont in ['W9','W10','W11','W12']:
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

for cont in ['W13','W14','W15','W16',]:
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

t=max(abs(np.array(axs[irow,icol].get_ylim())))
axs[irow,icol].set_ylim([-t,t])
axs[irow,icol].legend()
axs[irow,icol].grid()


In [ ]:
fig, axs = yu.getFigAxs(1,1,Lrow=16,Lcol=20)
tmin=0; tmax=16
irow=0; icol=0

spacing=0.04
i=0
for cont in ['n&pi+_p&pi0']:
    pi2Str='pi2=0_0_1'; mom=193
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

for cont in ['p&pi0_n&pi+']:
    pi2Str='pi2=0_0_1'; mom=193
    with h5py.File(path+'W_2pt.h5') as f:
        dat=np.real([f[cfg+'/12/'+pi2Str][cont][:,mom,0,0] for cfg in f.keys()])
        (mean,cov,err) = yu.jackknife(dat)
        tMean=mean[tmin:tmax]; tErr=err[tmin:tmax]
        axs[irow,icol].errorbar(np.arange(tmin,tmax)+i*spacing,tMean,tErr,fmt='o',elinewidth=2,capsize=6,mfc='white',label=cont)
        t=np.mean(dat,axis=0)
        print(cont)
        print(t)
    i+=1

t=max(abs(np.array(axs[irow,icol].get_ylim())))
axs[irow,icol].set_ylim([-t,t])
axs[irow,icol].legend()
axs[irow,icol].grid()
